# 0. Init

In [17]:
%cd /content/drive/MyDrive/NUS_MSBA/CS5344/CS5344 project/dataset

/content/drive/MyDrive/NUS_MSBA/CS5344/CS5344 project/dataset


In [18]:
!pip install pyspellchecker
!pip install nltk
!pip install langdetect

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

In [19]:
import csv
import os
import json
import argparse
import random
import re
import pandas as pd
import numpy as np

from tqdm import tqdm

# spelling
from spellchecker import SpellChecker

# nltk
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# language detector
from langdetect import detect
from langdetect import DetectorFactory
DetectorFactory.seed = 0

# 'here_for,'smoke','drink', 'match_age'
union_meta = ['scam','username','age','gender','location','ethnicity','occupation','marital_status','children','religion','sexual_orientation','sex', 'description']

scam_meta = ['email']

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [20]:
tqdm.pandas()

# 1. Clean-up

In [21]:
normal_profile = pd.read_csv('data.csv')
scam_profile1 = pd.read_csv('spam_output_female.csv')
scam_profile2 = pd.read_csv('spam_output_male.csv')
scam_profile = pd.concat([scam_profile1, scam_profile2])

In [22]:
# fix the scam description
def save_first_paragraph(text):
    if isinstance(text, str):
        text = text.split('IP')
        text = text[0]

        text = text.lower()
        text = text.split('ip address')
        text = text[0]

        text = text.split('message: – why is it a scam')
        text = text[0]

        text = text.split('message: –')
        text = text[0]

        text = text.split('why is it a scam')
        text = text[0]

        return text
    else:
        return None  # Return None if the cell is not a string

# Apply the function to the 'description' column
scam_profile['description'] = scam_profile['description'].progress_apply(save_first_paragraph)

scam_profile[['description']].dropna()

100%|██████████| 6116/6116 [00:00<00:00, 36098.92it/s]


,description
0,hi there i’m new here looking for a single man...
1,"am a cool headed,easy going ,honest,loving car..."
2,you can always be sure in my sincerity and hon...
3,i’m 5’9 and i’m an independent woman friendly ...
4,am quite.caring and loving woman. am family or...
...,...
3421,my name is richard. i am nice caring and lovin...
3422,hello how are you doing today ?well i really a...
3423,i am a man of diginity who have love and respe...
3424,"well what do i have to say, i am a person who ..."


In [23]:
# Create scam binary variable
normal_profile['scam'] = 0
scam_profile['scam'] = 1

# Standardise fields
normal_profile = normal_profile[union_meta]
scam_profile = scam_profile[union_meta]

# Union
all_profile = pd.concat([normal_profile, scam_profile])

# Convert into lowercase
all_profile = all_profile.astype(str)
all_profile = all_profile.applymap(lambda x: x.lower())

all_profile = all_profile.replace('nan', np.nan)

# Drop username = NaN
all_profile = all_profile.dropna(subset=['username'])

In [24]:
%time
spell = SpellChecker()
column_to_check = ['ethnicity','marital_status','religion']

def correct_spelling(text):
    if pd.notnull(text) and isinstance(text, str):
        words = text.split()
        correct_words = [spell.correction(word) if word is not None else '' for word in words]
        correct_words = [str(word) for word in correct_words if word is not None]
        return ' '.join(correct_words)
    elif text is None:
        return ''
    else:
        return text

for col in column_to_check:
    print(col)
    all_profile[col] = all_profile[col].progress_apply(correct_spelling)
    print('Done!')

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.06 µs
ethnicity


100%|██████████| 56906/56906 [00:14<00:00, 3860.87it/s]


Done!
marital_status


100%|██████████| 56906/56906 [00:05<00:00, 10460.85it/s]


Done!
religion


 84%|████████▍ | 47754/56906 [00:03<00:00, 12652.26it/s] 


KeyboardInterrupt: ignored

In [ ]:
all_profile.to_csv("all_profile_spelling.csv")

In [25]:
all_profile = pd.read_csv("all_profile_spelling.csv")
all_profile = all_profile.astype(str)

## Process attibutes

In [26]:
# List Unique value
def list_unique_values(df):
    for column in df.columns:
        unique_values = df[column].unique()
        count_values = len(unique_values)
        values_str = ', '.join(map(str, unique_values))
        print(f"{count_values} {column}: [{values_str}]")

list_unique_values(all_profile[union_meta[:-1]])

'''
1) remove wrong values and encode 0, 1 as nan
2) age: remove " y.o."
3) children:
   - i don’t have children >> no children
   - remove "i "
   - remove "have "
   - remove "children "
   - 1
'''

2 scam: [0, 1]
56643 username: [ausguy2013, kennethrw, luis2021, freespirit4ever, billyhill, naz_1, jaymnparty, edis43t, vic964, randygj, alex2345, rojopocoloco, 2023susan, cesar1878, bud61, rojo916, princeamon, globetrotter, junioreramos47, galla66, edgardor1001, edwin28, laura94, borjan, sylvester, susi_me, sectional, disas, toni2023, sol_97, bsharp1013, aalberts, josearteaga, mauro48, miiguel, jake39, sultando, fragola, kingsaturn, josecito18, krys83, josejose, feddy, lambo, hodge4575, mercedesre, omar1985, sunshineinca, blondeblueeyes, mariaisabel2009, carlospalma68, dorty_rios, oalberto, florinda, harrypadda, pan, dave4real, camachoblad, 7434don1, orazio, jose260559, triunfadora33, janata, jose966, nvx, trulysweetguy, daani, evavirg, linda71, manuelcasillo, lorenciya, morena23, raimermp, soltera65, jose1225, bennyrq, sparky, karl_pl, mikelo, fidel31, yovi19, roady, mauricio777, jhonal, serfa999, franny66, estefaniacr3, exson484, geovan82, angeluz, mike20850, gulnara77ya, operacion

'\n1) remove wrong values and encode 0, 1 as nan\n2) age: remove " y.o."\n3) children:\n   - i don’t have children >> no children\n   - remove "i "\n   - remove "have "\n   - remove "children "\n   - 1\n'

In [27]:
# Function to split 'x or y' values in specified columns
splitfields = ['occupation', 'ethnicity', 'marital_status']

temp_df = all_profile[all_profile[splitfields].progress_apply(lambda x: x.str.contains(' or ')).any(axis=1)]

# Delete original row from the full list
for col in splitfields:
  all_profile = all_profile[~all_profile[col].astype(str).str.contains(' or ')]

split_rows = []

for index, row in temp_df.iterrows():
    for field in splitfields:
        if ' or ' in row[field]:
            values = [val.strip() for val in row[field].split(' or ')]  # Splitting the row values
            for value in values:
                temp_row = row.copy()
                temp_row[field] = value.strip()
                split_rows.append(temp_row)

# Creating a new DataFrame from the list of split rows
split_df = pd.DataFrame(split_rows)

# Add the split row back
all_profile = pd.concat([all_profile,split_df])

100%|██████████| 3/3 [00:00<00:00, 39.80it/s]


In [28]:
# change age format
all_profile['age'] = all_profile['age'].astype(str).progress_apply(lambda x: x.replace(' y.o.', '')).astype(float)

# change 1, 0 into nan
encode_field = ['ethnicity', 'marital_status', 'religion', 'age', 'children', 'sexual_orientation']
for col in encode_field:
    all_profile[col] = all_profile[col].progress_apply(lambda x: np.nan if x == 1 or x == 0 else x)

# delete entry with wrongly filled value
delete_dict = {
               'ethnicity':['white', 'hispanic', 'mixed', 'asian', 'native american', 'black', 'middle eastern', 'pacific islander', 'other']
               ,'marital_status':['single', 'widowed', 'divorced', 'married', 'separated', 'in relationship']
               ,'religion': ['spiritual', 'christian', 'other', 'buddhist', 'atheist', 'muslim', 'jewish', 'hindu']
               ,'sexual_orientation': ['straight', 'bisexual', 'homosexual', 'sraight']
              }
for key, values in delete_dict.items():
    if len(values) > 0:
      all_profile.loc[~all_profile[key].isin(values), key] = np.nan

# replace 'nan' with np.nan
dall_profile = all_profile.replace('nan', np.nan)

100%|██████████| 56984/56984 [00:00<00:00, 821432.88it/s]


In [29]:
# change children
all_profile['children'].replace('i don’t have children', 'no children', inplace=True)
all_profile['children'].replace('don’t children', 'don’t want children', inplace=True)
all_profile['children'] = all_profile['children'].astype(str).progress_apply(lambda x: x.replace('i ', ''))
all_profile['children'] = all_profile['children'].astype(str).progress_apply(lambda x: x.replace('have ', ''))

100%|██████████| 56984/56984 [00:00<00:00, 411651.61it/s]


In [30]:
list_unique_values(all_profile[union_meta[:-1]])

2 scam: [0, 1]
56643 username: [ausguy2013, kennethrw, luis2021, freespirit4ever, billyhill, naz_1, jaymnparty, edis43t, vic964, randygj, alex2345, rojopocoloco, 2023susan, cesar1878, bud61, rojo916, princeamon, globetrotter, junioreramos47, galla66, edgardor1001, edwin28, laura94, borjan, sylvester, susi_me, sectional, disas, toni2023, sol_97, bsharp1013, aalberts, josearteaga, mauro48, miiguel, jake39, sultando, fragola, kingsaturn, josecito18, krys83, josejose, feddy, lambo, hodge4575, mercedesre, omar1985, sunshineinca, blondeblueeyes, mariaisabel2009, carlospalma68, dorty_rios, oalberto, florinda, harrypadda, pan, dave4real, camachoblad, 7434don1, orazio, jose260559, triunfadora33, janata, jose966, nvx, trulysweetguy, daani, evavirg, linda71, manuelcasillo, lorenciya, morena23, raimermp, soltera65, jose1225, bennyrq, sparky, karl_pl, mikelo, fidel31, yovi19, roady, mauricio777, jhonal, serfa999, franny66, estefaniacr3, exson484, geovan82, angeluz, mike20850, gulnara77ya, operacion

In [31]:
# bin age
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

# Define the labels for the bins
labels = ['1-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100']

# Use pd.cut to create the age bins
all_profile['age_bin'] = pd.cut(all_profile['age'], bins=bins, labels=labels, right=False)

In [32]:
all_profile

,Unnamed: 0,scam,username,age,gender,location,ethnicity,occupation,marital_status,children,religion,sexual_orientation,sex,description,age_bin
0,0,0,ausguy2013,50.0,male,"warrnambool vic 3280, australia",white,farmer/property development,single,want children,spiritual,straight,male,nan,51-60
1,1,0,kennethrw,54.0,male,"steele, al 35987, usa",white,pc tech,widowed,1-2 living elsewhere,christian,straight,male,nan,51-60
2,2,0,luis2021,48.0,male,"caracas, distrito capital, venezuela",hispanic,abogado/asesor,divorced,1-2 living with me,christian,straight,male,soy sensible cuando puedo apoyar a alguien lo ...,41-50
3,3,0,freespirit4ever,55.0,male,"sherman oaks, los angeles, ca, usa",mixed,engineer,single,no children,spiritual,straight,male,nan,51-60
4,4,0,billyhill,58.0,male,"los angeles, ca, usa",white,electrician,married,1-2 living with me,spiritual,straight,male,"i fall into the category of ""n more"" with dati...",51-60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56215,2733,1,ryandad8,48.0,male,"fremont, california, or fort worth, texas, uni...",NaN,antiquing,divorced,nan,NaN,NaN,male,i’m a single dad of a well mannered 4yr old bo...,41-50
56220,2738,1,marcusw670,52.0,male,"miami, florida, or los angeles, california, or...",NaN,nan,NaN,nan,NaN,NaN,male,"i am a mature,single man. i am 52 years old no...",51-60
56220,2738,1,marcusw670,52.0,male,"miami, florida, or los angeles, california, or...",NaN,nan,divorced,nan,NaN,NaN,male,"i am a mature,single man. i am 52 years old no...",51-60
56610,3129,1,lovelessmichaelklus,42.0,male,"marion or gary, indiana, united states",NaN,construction,divorced,nan,NaN,NaN,male,"i’m focused, sincere, caring,compasionate,dedi...",41-50


In [33]:
all_profile['location'].dropna()

0                          warrnambool vic 3280, australia
1                                    steele, al 35987, usa
2                     caracas, distrito capital, venezuela
3                       sherman oaks, los angeles, ca, usa
4                                     los angeles, ca, usa
                               ...                        
56215    fremont, california, or fort worth, texas, uni...
56220    miami, florida, or los angeles, california, or...
56220    miami, florida, or los angeles, california, or...
56610               marion or gary, indiana, united states
56610               marion or gary, indiana, united states
Name: location, Length: 56984, dtype: object

In [34]:
# remove accents
import unicodedata

def remove_accents(text):
    if isinstance(text, str):
        nfkd_form = unicodedata.normalize('NFKD', text)
        return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])
    else:
        return np.nan  # Return an empty string if the text is not a string

col_list = ['location', 'occupation']
for col in col_list:
  all_profile[col] = all_profile[col].progress_apply(remove_accents)

100%|██████████| 56984/56984 [00:00<00:00, 337879.55it/s]


In [35]:
all_profile['location'].dropna()

0                          warrnambool vic 3280, australia
1                                    steele, al 35987, usa
2                     caracas, distrito capital, venezuela
3                       sherman oaks, los angeles, ca, usa
4                                     los angeles, ca, usa
                               ...                        
56215    fremont, california, or fort worth, texas, uni...
56220    miami, florida, or los angeles, california, or...
56220    miami, florida, or los angeles, california, or...
56610               marion or gary, indiana, united states
56610               marion or gary, indiana, united states
Name: location, Length: 56984, dtype: object

In [36]:
all_profile.to_csv("processed_data.csv")

# 2. process occupation and description

In [41]:
all_profile = pd.read_csv("processed_data.csv")

In [42]:
all_profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56984 entries, 0 to 56983
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0.2        56984 non-null  int64  
 1   Unnamed: 0.1        56984 non-null  int64  
 2   Unnamed: 0          56984 non-null  int64  
 3   scam                56984 non-null  int64  
 4   username            56984 non-null  object 
 5   age                 56980 non-null  float64
 6   gender              56984 non-null  object 
 7   location            55956 non-null  object 
 8   ethnicity           52661 non-null  object 
 9   occupation          34584 non-null  object 
 10  marital_status      55041 non-null  object 
 11  children            51452 non-null  object 
 12  religion            51112 non-null  object 
 13  sexual_orientation  51451 non-null  object 
 14  sex                 56984 non-null  object 
 15  description         12568 non-null  object 
 16  age_

In [45]:
all_profile[['description']]

,description
0,NaN
1,NaN
2,NaN
3,NaN
4,"i fall into the category of ""n more"" with dati..."
...,...
56979,NaN
56980,"i am a mature,single man. i am 52 years old no..."
56981,"i am a mature,single man. i am 52 years old no..."
56982,"i’m focused, sincere, caring,compasionate,dedi..."


In [46]:
# detect english
def is_english(text):
    try:
        words = text.split()
        first_10_words = " ".join(words[:10])
        # print(first_10_words)
        lang = detect(first_10_words)
        # print(f"Detected language: {lang}")
        return lang == 'en'
    except:
        return False

all_profile.loc[~all_profile['description'].progress_apply(is_english), 'description'] = np.nan
all_profile[['description']]
all_profile.to_csv("processed_data.csv")

100%|██████████| 56984/56984 [02:04<00:00, 456.66it/s]


In [43]:
all_profile = pd.read_csv("processed_data.csv")

In [47]:
# remove stopwords
def remove_stopwords(text):
    if isinstance(text, str):  # Check if the text is a string
        stop_words = set(stopwords.words('english'))  # Set of English stopwords
        words = word_tokenize(text)
        filtered_words = [re.sub(r'[^a-zA-Z0-9\s]', '', word) for word in words if word.lower() not in stop_words]
        return ' '.join(filtered_words)
    else:
        return np.nan  # Return an empty string if the text is not a string

all_profile['description_set'] = all_profile['description'].progress_apply(remove_stopwords)
all_profile[['description_set']]

100%|██████████| 56984/56984 [00:12<00:00, 4494.32it/s]


,description_set
0,NaN
1,NaN
2,NaN
3,NaN
4,fall category n dating n married looking a...
...,...
56979,NaN
56980,mature single man 52 years old lot tell one...
56981,mature single man 52 years old lot tell one...
56982,focused sincere caring compasionate dedic...


In [ ]:
# # spelling
# %time
# spell = SpellChecker()

# def correct_spelling(text):
#     if pd.notnull(text) and isinstance(text, str):
#         words = text.split()
#         correct_words = [spell.correction(word) if word is not None else '' for word in words]
#         correct_words = [str(word) for word in correct_words if word is not None]
#         return ' '.join(correct_words)
#     elif text is None:
#         return ''
#     else:
#         return text

# text_col = ['description_set']
# for col in text_col:
#     print(col)
#     all_profile[col] = all_profile[col].progress_apply(correct_spelling)
#     # all_profile.to_csv("processed_data.csv")
#     print('Done!')

In [48]:
# stem
def stem_words(text):
    if isinstance(text, str):  # Check if the text is a string
        porter = PorterStemmer()
        token_words = word_tokenize(text)
        stem_sentence = [porter.stem(word) for word in token_words]
        return ' '.join(stem_sentence)
    else:
        return np.nan  # Return an empty string if the text is not a string

all_profile['description_stem'] = all_profile['description_set'].progress_apply(stem_words)
all_profile[['description_stem']]

100%|██████████| 56984/56984 [00:17<00:00, 3320.91it/s]


,description_stem
0,NaN
1,NaN
2,NaN
3,NaN
4,fall categori n date n marri look anyth els he...
...,...
56979,NaN
56980,matur singl man 52 year old lot tell one reall...
56981,matur singl man 52 year old lot tell one reall...
56982,focus sincer care compasion dedic finish start...


In [49]:
def extract_unique_words(text):
    if isinstance(text, str):
      return set(text.split())
    else:
        return np.nan

all_profile['description_set'] = all_profile['description_set'].progress_apply(extract_unique_words)
all_profile['description_stem_set'] = all_profile['description_stem'].progress_apply(extract_unique_words)

100%|██████████| 56984/56984 [00:00<00:00, 182723.81it/s]


In [50]:
all_profile[['description','description_set', 'description_stem', 'description_stem_set']].dropna()

,description,description_set,description_stem,description_stem_set
4,"i fall into the category of ""n more"" with dati...","{school, helpful, always, label, anything, acc...",fall categori n date n marri look anyth els he...,"{friend, school, label, walk, accept, look, ot..."
6,"i am incredibly social (& thus, for the most p...","{new, nice, im, friends, note, anyone, leader,...",incred social thu part like meet new peopl lik...,"{friend, new, nice, im, other, note, leader, m..."
7,"i enjoy gardening, fishing, camping, hiking an...","{flexible, hiking, degree, games, hopefully, u...",enjoy garden fish camp hike go walk also like ...,"{trustworthi, engin, walk, orient, look, woman..."
14,i live in southern maryland.just moved back af...,"{lady, currentinactive, women, edges, andrew, ...",live southern marylandjust move back live flor...,"{talk, women, d, andrew, attemptedi, gener, ge..."
19,i am a strong woman with a soft character and ...,"{capable, loving, heart, kind, character, humo...",strong woman soft charact kind heart faith gen...,"{heart, kind, humor, gentl, thing, strong, cap..."
...,...,...,...,...
56975,"as an engineer, i’m a deep and innovative thin...","{kind, stories, slow, energy, widely, life, fr...",engin deep innov thinker look underli pattern ...,"{underli, friend, kind, engin, slow, look, lif..."
56980,"i am a mature,single man. i am 52 years old no...","{52, lady, hello, dear, nice, really, profile,...",matur singl man 52 year old lot tell one reall...,"{52, hello, dear, nice, profil, great, tell, f..."
56981,"i am a mature,single man. i am 52 years old no...","{52, lady, hello, dear, nice, really, profile,...",matur singl man 52 year old lot tell one reall...,"{52, hello, dear, nice, profil, great, tell, f..."
56982,"i’m focused, sincere, caring,compasionate,dedi...","{friend, happy, new, enter, michaelklus606, he...",focus sincer care compasion dedic finish start...,"{friend, new, enter, michaelklus606, hello, pr..."


In [51]:
all_profile.to_csv("processed_data.csv")

In [52]:
all_profile

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,scam,username,age,gender,location,ethnicity,occupation,marital_status,children,religion,sexual_orientation,sex,description,age_bin,description_set,description_stem,description_stem_set
0,0,0,0,0,ausguy2013,50.0,male,"warrnambool vic 3280, australia",white,farmer/property development,single,want children,spiritual,straight,male,NaN,51-60,NaN,NaN,NaN
1,1,1,1,0,kennethrw,54.0,male,"steele, al 35987, usa",white,pc tech,widowed,1-2 living elsewhere,christian,straight,male,NaN,51-60,NaN,NaN,NaN
2,2,2,2,0,luis2021,48.0,male,"caracas, distrito capital, venezuela",hispanic,abogado/asesor,divorced,1-2 living with me,christian,straight,male,NaN,41-50,NaN,NaN,NaN
3,3,3,3,0,freespirit4ever,55.0,male,"sherman oaks, los angeles, ca, usa",mixed,engineer,single,no children,spiritual,straight,male,NaN,51-60,NaN,NaN,NaN
4,4,4,4,0,billyhill,58.0,male,"los angeles, ca, usa",white,electrician,married,1-2 living with me,spiritual,straight,male,"i fall into the category of ""n more"" with dati...",51-60,"{school, helpful, always, label, anything, acc...",fall categori n date n marri look anyth els he...,"{friend, school, label, walk, accept, look, ot..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56979,56979,56215,2733,1,ryandad8,48.0,male,"fremont, california, or fort worth, texas, uni...",NaN,antiquing,divorced,NaN,NaN,NaN,male,NaN,41-50,NaN,NaN,NaN
56980,56980,56220,2738,1,marcusw670,52.0,male,"miami, florida, or los angeles, california, or...",NaN,NaN,NaN,NaN,NaN,NaN,male,"i am a mature,single man. i am 52 years old no...",51-60,"{52, lady, hello, dear, nice, really, profile,...",matur singl man 52 year old lot tell one reall...,"{52, hello, dear, nice, profil, great, tell, f..."
56981,56981,56220,2738,1,marcusw670,52.0,male,"miami, florida, or los angeles, california, or...",NaN,NaN,divorced,NaN,NaN,NaN,male,"i am a mature,single man. i am 52 years old no...",51-60,"{52, lady, hello, dear, nice, really, profile,...",matur singl man 52 year old lot tell one reall...,"{52, hello, dear, nice, profil, great, tell, f..."
56982,56982,56610,3129,1,lovelessmichaelklus,42.0,male,"marion or gary, indiana, united states",NaN,construction,divorced,NaN,NaN,NaN,male,"i’m focused, sincere, caring,compasionate,dedi...",41-50,"{friend, happy, new, enter, michaelklus606, he...",focus sincer care compasion dedic finish start...,"{friend, new, enter, michaelklus606, hello, pr..."
